# Project: Keccak hash function
Implement a Keccak hash function - as an example of a sponge structure and analyze its security.

<div style="text-align: right">
Karolina Buchnat<br>
Aleksander Malcew
</div>

## State initialization

In [51]:
import numpy as np

def initialize_state():
    """
    Initializes an empty Keccak state.
    
    Keccak uses a state consisting of 5x5x64 bits, totaling 1600 bits.
    This function initializes the state as a three-dimensional array filled with zeros.
    
    Returns:
        numpy.ndarray: A 5x5x64 array representing the initial state of Keccak, filled with zeros.
    """
    state = np.zeros((5, 5, 64), dtype=np.uint8)
    return state

# Initialize the state
state = initialize_state()
print("State shape:", state.shape)
print("State data type:", state.dtype)
print("State array:\n", state)


State shape: (5, 5, 64)
State data type: uint8
State array:
 [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


## Padding

In [55]:
def pad10star1(x, m):
    """
    Implements the pad10*1 padding function operating on binary values.

    This function applies the pad10*1 padding rule, which is used to extend the message so that
    its length (in bits) plus the length of the padding is a positive multiple of the bitrate x.
    
    The padding consists of a '1' bit, followed by a number of '0' bits, and ends with a '1' bit.
    
    Parameters:
        x (int): A positive integer representing the bitrate (r) of the Keccak algorithm.
        m (int): A non-negative integer indicating the length of the message in bits.
    
    Returns:
        list: A list of bits representing the padding P, such that the length of m + len(P) is a positive multiple of x.
    """
    j = (-m - 2) % x
    P = [1] + [0] * j + [1]
    
    return P

# Example usage
x = 1088  # Example bitrate for SHA3-256
m = 1024  # Example message length in bits

padding = pad10star1(x, m)
print(f"Padding for a message of length {m} bits and bitrate {x} is: {padding[:10]}... (first 10 bits)")
print(f"Padding length: {len(padding)} bits")


Padding for a message of length 1024 bits and bitrate 1088 is: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]... (first 10 bits)
Padding length: 64 bits


## Step mapping

### Theta (θ)

In [56]:
def theta(A):
    """
    Performs the Theta step of the Keccak permutation.

    The Theta step is one of the five step mappings in the Keccak-f permutation.
    It operates on the state array A and mixes the bits across the five lanes in each of the 64-bit words.

    Parameters:
        A (numpy.ndarray): The current state of the Keccak algorithm, a 5x5x64 array of bits represented as 64-bit words.
    
    Returns:
        numpy.ndarray: The updated state array after applying the Theta step.
    """
    w = 64  # Word size for Keccak-f[1600]
    C = np.zeros((5, w), dtype=np.uint64)  # Intermediate array C
    D = np.zeros((5, w), dtype=np.uint64)  # Intermediate array D
    
    # Processing is done on bits, but using NumPy, we operate on 64-bit words
    for x in range(5):
        # XOR of all the bits in the lanes
        C[x, :] = A[x, 0, :] ^ A[x, 1, :] ^ A[x, 2, :] ^ A[x, 3, :] ^ A[x, 4, :]
    
    for x in range(5):
        # Calculate the D array based on C
        D[x, :] = C[(x-1) % 5, :] ^ np.roll(C[(x+1) % 5, :], -1)
    
    for x in range(5):
        for y in range(5):
            # Apply the D array to the state A
            A[x, y, :] ^= D[x, :]
    
    return A


### Rho (ρ)

In [57]:
def rho(A):
    """
    Performs the Rho step of the Keccak permutation.
    
    The Rho step is responsible for bit-wise rotations within the lanes of the state array.
    Each lane is rotated by an offset specific to its position (x, y) in the state array.
    
    Parameters:
        A (numpy.ndarray): The current state of the Keccak algorithm, a 5x5x64 array of bits.
    
    Returns:
        numpy.ndarray: The updated state array after applying the Rho step.
    """
    w = 64  # Word size for Keccak-f[1600]
    # Initialize A' (A_prime) as a copy of A
    A_prime = np.copy(A)
    
    # Step 1: A'[0, 0, z] = A[0, 0, z]
    # (this operation is already done by initializing A_prime as a copy of A)
    
    # Step 2: Set the initial values (x, y)
    x, y = 1, 0
    
    # Offset table for the Rho function, according to the specification table
    offsets = [
        [0, 36, 3, 41, 18],
        [1, 44, 10, 45, 2],
        [62, 6, 43, 15, 61],
        [28, 55, 25, 21, 56],
        [27, 20, 39, 8, 14]
    ]
    
    # Step 3: Perform rotation for each t from 0 to 23
    for t in range(24):
        # Step 3a: Rotate bits
        offset = offsets[x][y]
        A_prime[x, y] = np.roll(A[x, y], -offset)
        
        # Step 3b: Calculate new values (x, y)
        x, y = y, (2 * x + 3 * y) % 5
    
    return A_prime


### Pi (π)

In [59]:
def pi(A):
    """
    Performs the Pi step of the Keccak permutation.

    The Pi step rearranges the positions of the lanes in the state array. It is a permutation 
    that transposes the axes of the array to ensure diffusion of the state bits across the array.
    
    Parameters:
        A (numpy.ndarray): The current state of the Keccak algorithm, a 5x5x64 array of bits.
    
    Returns:
        numpy.ndarray: The updated state array after applying the Pi step.
    """
    # Word size in Keccak, w = 64 for Keccak-f[1600]
    w = 64
    # Initialize A' (A_prime) as an empty array with the same dimensions as A
    A_prime = np.zeros_like(A)
    
    # Processing according to the specification of the Pi function
    for x in range(5):
        for y in range(5):
            for z in range(w):
                newX = (x + 3 * y) % 5
                newY = x
                A_prime[newX, newY, z] = A[x, y, z]
    
    return A_prime


### Chi (χ)

In [60]:
def chi(A):
    """
    Performs the Chi step of the Keccak permutation.

    The Chi step is a non-linear step that operates on each row of the state array independently.
    It combines each bit with two other bits in the same row, using a bitwise XOR and AND operations,
    to contribute to the confusion aspect of the cipher.

    Parameters:
        A (numpy.ndarray): The current state of the Keccak algorithm, a 5x5x64 array of bits.
    
    Returns:
        numpy.ndarray: The updated state array after applying the Chi step.
    """
    # Word size in Keccak, w = 64 for Keccak-f[1600]
    w = 64
    # Initialize A' (A_prime) as an empty array with the same dimensions as A
    A_prime = np.zeros_like(A)
    
    # Processing according to the specification of the Chi function
    for x in range(5):
        for y in range(5):
            for z in range(w):
                # For each bit, apply the Chi transformation
                A_prime[x, y, z] = A[x, y, z] ^ ((A[(x+1) % 5, y, z] ^ 1) & A[(x+2) % 5, y, z])
    
    return A_prime


### Iota (ι)

In [62]:
from math import log2

def rc(t):
    """
    Computes the round constant for the Iota step in the Keccak algorithm.

    This function is based on a linear feedback shift register (LFSR) to generate
    a sequence of bits that are used as round constants in the Iota step of the Keccak permutation.

    Parameters:
        t (int): The step index for which the round constant is generated.
    
    Returns:
        int: A single bit (0 or 1) representing the round constant for the given step index.
    """
    if t % 255 == 0:
        return 1
    R = [1] + [0]*7
    for i in range(1, t % 255):
        R = [0] + R
        R[0] = R[0] ^ R[8]
        R[4] = R[4] ^ R[8]
        R[5] = R[5] ^ R[8]
        R[6] = R[6] ^ R[8]
        R = R[:-1]
    return R[0]

def iota(A, ir):
    """
    Performs the Iota step of the Keccak permutation.

    The Iota step is the final step in one round of the Keccak permutation. It modifies the state
    by XORing it with a round constant. The round constants are derived from the output of the
    `rc` function, which depends on the round index.

    Parameters:
        A (numpy.ndarray): The current state of the Keccak algorithm, a 5x5x64 array of bits.
        ir (int): The round index, used to compute the round constant.
    
    Returns:
        numpy.ndarray: The updated state array after applying the Iota step.
    """
    w = 64  # Word size for Keccak-f[1600]
    l = int(log2(w))
    RC = [0] * w
    
    # Compute RC
    for j in range(l + 1):
        RC[(1 << j) - 1] = rc(j + 7*ir)
    
    # Modify A[0, 0, z] by XORing with RC[z]
    A_prime = np.copy(A)
    for z in range(w):
        A_prime[0, 0, z] ^= RC[z]
    
    return A_prime
    

## KECCAK-p[b, nr]

In [63]:
def string_to_state_array(S):
    """
    Converts a bit string S into the state array A for the Keccak algorithm.

    This function transforms a list of bits representing the input message or data into
    the internal state format used by the Keccak hash function. The Keccak state consists
    of a 5x5 array of 64-bit words, totaling 1600 bits.

    Parameters:
        S (list): A list of bits (0s and 1s) with a length of b = 1600, representing the input data.
    
    Returns:
        numpy.ndarray: A three-dimensional state array A of shape (5, 5, 64), corresponding to the Keccak state.
    """
    # Dimensions of the state array
    w = 64  # word size
    depth = w
    rows, cols = 5, 5  # dimensions of the state array
    
    # Initialize the state array
    A = np.zeros((rows, cols, depth), dtype=np.uint8)
    
    # Convert S into A
    for x in range(5):
        for y in range(5):
            for z in range(w):
                # Calculate the bit index for the flat list and map it to the 3D state array
                bit_index = w * (5 * y + x) + z
                A[x, y, z] = S[bit_index]
    
    return A

# Example usage:
# Generate an example bit string S as a list of 1600 alternating zeros and ones
S_example = [1 if i % 2 == 0 else 0 for i in range(1600)]
A_example = string_to_state_array(S_example)

print("State A[0, 0, :] (first 64 bits):", A_example[0, 0, :])


State A[0, 0, :] (first 64 bits): [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]


In [64]:
def Rnd(A, ir):
    """
    Performs a single round of the Keccak transformation on the given state A with the round index ir.

    This function sequentially applies all the five step mappings (theta, rho, pi, chi, iota) that constitute
    a single round of the Keccak permutation. These transformations progressively mix and permute the bits
    in the state array to produce a new state that is computationally indistinguishable from random to someone
    who doesn't know the input.

    Parameters:
        A (numpy.ndarray): The current state of the Keccak algorithm, represented as a three-dimensional NumPy array.
        ir (int): The index of the current round, used in the iota (ι) step to introduce round-dependent variability.
    
    Returns:
        numpy.ndarray: The updated state array after applying one round of the Keccak transformation.
    """
    A = theta(A)
    A = rho(A)
    A = pi(A)
    A = chi(A)
    A = iota(A, ir)
    
    return A


In [65]:
def state_array_to_string(A):
    """
    Converts the state array A into a bit string S.

    This function maps the three-dimensional state array used in the Keccak algorithm
    back into a linear list of bits. This is particularly useful for extracting the 
    hash value after the final state has been computed, or for any intermediate steps
    that require the state to be analyzed or output in a linear format.

    Parameters:
        A (numpy.ndarray): The three-dimensional state array (e.g., in NumPy format) representing the Keccak state.
    
    Returns:
        list: The bit string S as a list of bits.
    """
    w = 64  # Word size in Keccak-f[1600]
    S = []
    
    # Iterate through the state array as per the specification
    for y in range(5):
        for x in range(5):
            for z in range(w):
                # Extract each bit and append to the list S
                bit = A[x, y, z]
                S.append(bit)
    
    return S

# Example usage:
A_example = np.random.randint(2, size=(5, 5, 64), dtype=np.uint8)
S_converted = state_array_to_string(A_example)

print("First 100 bits of S:", S_converted[:100])


First 100 bits of S: [0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1]


In [66]:
def KECCAK_p(S, nr=24):
    """
    Executes the full KECCAK-p permutation for the Keccak algorithm.

    This function applies the complete permutation process of the Keccak algorithm, 
    which involves converting the input bit string into the state array, performing 
    a specified number of permutation rounds, and then converting the final state array 
    back into a bit string.

    Parameters:
        S (list): The input bit string as a list of bits with a length of b = 1600.
        nr (int): The number of permutation rounds.
    
    Returns:
        list: The output bit string S′ as a list of bits with a length of b.
    """
    # Step 1: Convert S into the state array A
    A = string_to_state_array(S)
    
    # Step 2: Perform the permutation rounds
    for ir in range(nr):
        A = Rnd(A, ir)
    
    # Step 3: Convert A back into the output bit string S′
    S_prime = state_array_to_string(A)
    
    return S_prime

# Example usage:
S_example = np.random.randint(2, size=1600).tolist()
S_prime_example = KECCAK_p(S_example, nr=24)

print("Length of S_prime:", len(S_prime_example))
print("First 100 bits of S_prime:", S_prime_example[:100])


Length of S_prime: 1600
First 100 bits of S_prime: [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0]


## Sponge construction

In [67]:
def SPONGE(f, pad, r, N, d):
    """
    Implements the sponge function for the Keccak algorithm.

    The sponge function absorbs the input bit string into the state, processes it through
    the permutation function, and then squeezes out the output bit string of desired length.

    Parameters:
        f (function): The permutation function.
        pad (function): The padding function.
        r (int): The bitrate, which determines the amount of data processed in each permutation round.
        N (list): The input bit string as a list of bits.
        d (int): The desired length of the output bit string Z.
    
    Returns:
        list: The output bit string Z of length d.
    """
    # Step 1: Pad the input string
    P = N + pad(r, len(N))
    
    # Step 2: Calculate the number of blocks
    n = len(P) // r
    
    # Step 3: Calculate c = b - r
    c = 1600 - r  # For Keccak, b = 1600
    
    # Step 4: Split P into blocks P0, …, Pn-1
    blocks = [P[i*r:(i+1)*r] for i in range(n)]
    
    # Step 5: Initialize the state S
    S = [0] * 1600
    
    # Step 6: Absorption phase
    for Pi in blocks:
        S = list(map(lambda x, y: x ^ y, S[:r] + [0]*c, Pi + [0]*c))
        S = f(S)
    
    # Step 7 and 8: Initialize Z and squeeze the first block
    Z = []
    Z.extend(S[:r])
    
    # Step 9 and 10: Continue squeezing until reaching the desired length d
    while len(Z) < d:
        S = f(S)
        Z.extend(S[:r])
    
    return Z[:d]

# Example usage
N = [1] * 1024  # Example input string N as a list of 1024 bits set to 1
d = 256  # Desired length of the output bit string
r = 1088  # Bitrate for SHA3-256

# Call the SPONGE function
Z = SPONGE(KECCAK_p, pad10star1_binary, r, N, d)
print("Output of the SPONGE function (first 64 bits):", Z[:64])
print("Length of the output string:", len(Z))


Output of the SPONGE function (first 64 bits): [1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
Length of the output string: 256


## Auxilary functions

In [68]:
def text_to_bits(text, encoding='utf-8'):
    """
    Converts text into a list of bits.

    Parameters:
        text (str): The text to be converted.
        encoding (str): The encoding used to convert the text into bytes. Defaults to 'utf-8'.
    
    Returns:
        list: A list of bits (0s and 1s) representing the text.
    """
    bits = []
    byte_array = text.encode(encoding)
    for byte in byte_array:
        bits.extend([int(bit) for bit in format(byte, '08b')])
    return bits

def bits_to_bytes(bits):
    """
    Converts a list of bits into a byte array.

    Parameters:
        bits (list): A list of bits (0s and 1s).
    
    Returns:
        bytes: A byte array corresponding to the list of bits.
    """
    bytes_list = [int(''.join(map(str, bits[i:i+8])), 2) for i in range(0, len(bits), 8)]
    return bytes(bytes_list)

def bytes_to_hex(byte_array):
    """
    Converts a byte array into a hexadecimal string.

    Parameters:
        byte_array (bytes): The byte array to be converted.
    
    Returns:
        str: A hexadecimal string representing the byte array.
    """
    return byte_array.hex()


## Keccak-256

In [69]:
def KECCAK_256(N):
    """
    Generates a SHA3-256 hash for the given input string N.
    
    This function takes an input bit string and processes it through the Keccak sponge construction
    to produce a SHA3-256 hash. The sponge construction involves padding the input, absorbing it into
    the Keccak state, and then squeezing out the hash value.

    Parameters:
        N (list): The input bit string as a list of bits.
    
    Returns:
        str: The SHA3-256 hash as a hexadecimal string.
    """
    b = 1600  # Total state length for Keccak
    c = 512   # Capacity for SHA3-256
    r = b - c  # Bitrate for SHA3-256
    d = 256   # Hash length for SHA3-256 in bits

    # Apply the sponge construction to get the hash bits
    hash_bits = SPONGE(KECCAK_p, pad10star1_binary, r, N, d)
    # Convert the hash bits to a hexadecimal string
    hash_hex = bytes_to_hex(bits_to_bytes(hash_bits))
    return hash_hex

# Example usage
text = "some"
# Convert the text to bits
N = text_to_bits(text)

# Generate the SHA3-256 hash
hash_256 = KECCAK_256(N)
print("SHA3-256 hash:", hash_256)


SHA3-256 hash: c27fed99271e7ab49a2a0d916b29d5bde21d9f80db7e194d36094ab7c5a454db


## Check correctness

In [70]:
import hashlib

def keccak_256_hash(text):
    # Konwersja tekstu na bajty
    text_bytes = text.encode('utf-8')
    
    # Użycie hashlib do wygenerowania skrótu Keccak-256
    hash_obj = hashlib.new('sha3_256', text_bytes)
    hash_hex = hash_obj.hexdigest()
    
    return hash_hex

# Przykład użycia
text = "some"
hash_result = keccak_256_hash(text)
print("Keccak-256 hash:", hash_result)

Keccak-256 hash: 7b080823f92e02130bdea91bd0f85eb8503b4b15f36a4a89f1698d087106eca9
